# Checkpoint Objective
This dataset contains banking marketing campaign data and we can use it to optimize marketing campaigns to attract more customers to term deposit subscription.

What is a Term Deposit?

A Term deposit is a deposit that a bank or a financial institution offers with a fixed rate (often better than just opening deposit account) in which your money will be returned back at a specific maturity time.

#### 1. Import your dataset and perform initial analysis and visualization

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.metrics import Accuracy

In [2]:
# Load and view the data 
df = pd.read_csv('31.Bank_AmaniYch.csv')
dt = df.copy()
dt

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [3]:
# Get general information about the data
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


In [4]:
# Summary statistics on numerical features
dt.describe()

,age,balance,day,duration,campaign,pdays,previous
count,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000
mean,41.231948,1528.538524,15.658036,371.993818,2.508421,51.330407,0.832557
std,11.913369,3225.413326,8.420740,347.128386,2.722077,108.758282,2.292007
min,18.000000,-6847.000000,1.000000,2.000000,1.000000,-1.000000,0.000000
25%,32.000000,122.000000,8.000000,138.000000,1.000000,-1.000000,0.000000
50%,39.000000,550.000000,15.000000,255.000000,2.000000,-1.000000,0.000000
75%,49.000000,1708.000000,22.000000,496.000000,3.000000,20.750000,1.000000
max,95.000000,81204.000000,31.000000,3881.000000,63.000000,854.000000,58.000000


In [5]:
# Visualizations on numerical data
# Histogram of the 'age' column
fig_age = px.histogram(dt, x='age', nbins=20, title='Distribution of Age')
fig_age.show()

# Scatter plot between 'age' and 'balance'
fig_scatter = px.scatter(dt, x='age', y='balance', title='Scatter Plot: Age vs. Balance')
fig_scatter.show()

In [6]:
# Value counts of object features
object_features = dt.select_dtypes(include='object').columns.tolist()

for feature in object_features:
    print(f"Value counts for '{feature}':")
    print(dt[feature].value_counts())
    print("-" * 30)

Value counts for 'job':
job
management       2566
blue-collar      1944
technician       1823
admin.           1334
services          923
retired           778
self-employed     405
student           360
unemployed        357
entrepreneur      328
housemaid         274
unknown            70
Name: count, dtype: int64
------------------------------
Value counts for 'marital':
marital
married     6351
single      3518
divorced    1293
Name: count, dtype: int64
------------------------------
Value counts for 'education':
education
secondary    5476
tertiary     3689
primary      1500
unknown       497
Name: count, dtype: int64
------------------------------
Value counts for 'default':
default
no     10994
yes      168
Name: count, dtype: int64
------------------------------
Value counts for 'housing':
housing
no     5881
yes    5281
Name: count, dtype: int64
------------------------------
Value counts for 'loan':
loan
no     9702
yes    1460
Name: count, dtype: int64
----------------------

#### 2. Clean the data

In [9]:
# Separate the features (X) and the target variable (y)
X = dt.drop('deposit', axis=1)
y = dt['deposit']

# Encode target variable to numerical format
label_encoder = LabelEncoder()
dt['deposit'] = label_encoder.fit_transform(y)

# One-hot encode categorical features 
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize/Scale numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### 3. Build a  deep learning network to predict the marketing campaign outcome ("deposit" either Yes or No)

In [10]:
# Initialization
batch_size = 32
epochs = 100 
learning_rate = 0.001

# Build the deep learning model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = learning_rate), metrics=['binary_accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/100
280/280 [==============================] - 2s 4ms/step - loss: 0.4388 - binary_accuracy: 0.8027 - val_loss: 0.3909 - val_binary_accuracy: 0.8253
Epoch 2/100
280/280 [==============================] - 1s 3ms/step - loss: 0.3715 - binary_accuracy: 0.8418 - val_loss: 0.3800 - val_binary_accuracy: 0.8285
Epoch 3/100
280/280 [==============================] - 1s 3ms/step - loss: 0.3522 - binary_accuracy: 0.8562 - val_loss: 0.3680 - val_binary_accuracy: 0.8374
Epoch 4/100
280/280 [==============================] - 1s 3ms/step - loss: 0.3371 - binary_accuracy: 0.8584 - val_loss: 0.3692 - val_binary_accuracy: 0.8334
Epoch 5/100
280/280 [==============================] - 1s 2ms/step - loss: 0.3312 - binary_accuracy: 0.8622 - val_loss: 0.3703 - val_binary_accuracy: 0.8433
Epoch 6/100
280/280 [==============================] - 1s 3ms/step - loss: 0.3208 - binary_accuracy: 0.8663 - val_loss: 0.3681 - val_binary_accuracy: 0.8442
Epoch 7/100
280/280 [==============================] - 1s 

In [11]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

70/70 [==============================] - 0s 2ms/step - loss: 0.9716 - binary_accuracy: 0.8159
Test Loss: 0.9716227054595947
Test Accuracy: 0.8159427046775818


In [12]:
# predict the marketing campaign outcome ("deposit" either Yes or No)
X = X.astype('float32')
pred = model.predict(X)

# Get the predicted class labels (0 or 1) based on the threshold (e.g., 0.5)
pred_classes = (pred >= 0.5).astype(int)

# Map the predicted class labels to 'Yes' or 'No'
predicted_labels = np.where(pred_classes == 1, 'yes', 'no')

# Add the predicted labels to the original DataFrame as a new column
df['predicted_deposit'] = predicted_labels

# Display the DataFrame with the predicted labels
df

349/349 [==============================] - 1s 1ms/step


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,predicted_deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no,yes
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no,yes
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no,yes
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no,yes
